In [ ]:
# load relevant packages
import datetime
import dateutil.parser
import math
import ast

In [ ]:
# load library to upload data in colab
from google.colab import files
# call function to select file to upload
uploaded = files.upload()

Saving data_ukraine conflict lang_en_2022_01_02_4500.csv to data_ukraine conflict lang_en_2022_01_02_4500.csv


In [ ]:
# import io to get access to io functions
import io
# load pandas to use its functions
import pandas as pd
# read uploaded file
d = pd.read_csv(io.BytesIO(uploaded['data_ukraine conflict lang_en_2022_01_02_4500.csv']), dtype=str)
# Dataset is now stored in a Pandas Dataframe

In [ ]:
# store topic theme
topic = "ukraine conflict"

In [ ]:
# test length
len(d)

4406

In [ ]:
# check dataframe
d

,Unnamed: 0,source,conversation_id,text,created_at,lang,in_reply_to_user_id,reply_settings,author_id,id,referenced_tweets,public_metrics,geo,withheld
0,0,Twitter for iPhone,1477707348969988097,deal that temporarily resolved the long-standi...,2022-01-02T19:03:19.000Z,en,1179464402791219206,everyone,1179464402791219206,1477717156959858693,"[{'type': 'replied_to', 'id': '147771560219847...","{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,NaN
1,1,Twitter Web App,1477716157264121856,US pledges to help Ukraine shore up border wit...,2022-01-02T18:59:21.000Z,en,NaN,everyone,1251224826984804352,1477716157264121856,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,NaN
2,2,Twitter Web App,1477715730376400901,RT @EuromaidanPR: Russian hybrid forces launch...,2022-01-02T18:57:39.000Z,en,NaN,everyone,950864773087932416,1477715730376400901,"[{'type': 'retweeted', 'id': '1477555636976107...","{'retweet_count': 30, 'reply_count': 0, 'like_...",NaN,NaN
3,3,Twitter for iPhone,1477714726511472641,Russia-Ukraine Conflict Lies in the Bones of a...,2022-01-02T18:53:40.000Z,en,NaN,everyone,104700877,1477714726511472641,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,NaN
4,4,Twitter Web App,1477711547707404291,They're looking for his bones.\nThe Russia-Ukr...,2022-01-02T18:41:02.000Z,en,NaN,everyone,1037420964320104450,1477711547707404291,NaN,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4401,6,Twitter for Android,1475189647252807684,RT @PrplScrubsCHPN: @RonFilipkowski @Judson4Co...,2021-12-26T19:39:54.000Z,en,NaN,everyone,2787089534,1475189647252807684,"[{'type': 'retweeted', 'id': '1475165931810738...","{'retweet_count': 25, 'reply_count': 0, 'like_...",NaN,NaN
4402,7,Twitter for iPad,1475189268989554690,RT @SkyNews: Speaking at his annual news confe...,2021-12-26T19:38:24.000Z,en,NaN,everyone,3053193701,1475189268989554690,"[{'type': 'retweeted', 'id': '1474021978109616...","{'retweet_count': 28, 'reply_count': 0, 'like_...",NaN,NaN
4403,8,Twitter for Android,1475157803157409803,@Morearighty It's easy. I'm a Russian immigran...,2021-12-26T19:35:44.000Z,en,939600814858960896,everyone,122756897,1475188601403150336,"[{'type': 'replied_to', 'id': '147516882233212...","{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,NaN
4404,9,Twitter for Android,1475188408427421701,RT @France24_en: @RobParsonsF24 The Wagner Gro...,2021-12-26T19:34:58.000Z,en,NaN,everyone,1400022519613214720,1475188408427421701,"[{'type': 'retweeted', 'id': '1474387385848893...","{'retweet_count': 19, 'reply_count': 0, 'like_...",NaN,NaN


In [ ]:
# test - check row
d['text'][1]

"US pledges to help Ukraine shore up border with Russia, Belarus: report https://t.co/Emuk7p4LaN via @Yahoo PUTIN THINKS THE U.S. WONT DEFEND UKRAINE'S BORDER IF THEY ATTACK. THE U.S ISN'T IN ANY CONFLICT RIGHT NOW AND WE NEED ANOTHER WAR FOR THE MEN WHO CAME BACK HOME!"

In [ ]:
# test - check column
d['author_id']

0       1179464402791219206
1       1251224826984804352
2        950864773087932416
3                 104700877
4       1037420964320104450
               ...         
4401             2787089534
4402             3053193701
4403              122756897
4404    1400022519613214720
4405     712345202971643906
Name: author_id, Length: 4406, dtype: object

In [ ]:
# loop over the loaded dataframe and perform some required preporcessings for further usages
# for instance, the Twitter API stores sometimes a list in a field resulting in deep table structures
# however, it is not desired to have these deep structures in later stages and this has to be done for all obtained data
# thus, this flattening should be done for all loaded Twitter data
# new data row is stored to new dataframe df
# here the preprocessed dataframe of d is then stored
df = pd.DataFrame()

for index, row in d.iterrows():
    # We will create a variable for each since some of the keys might not exist for some tweets
    # So we will account for that

    # 0. Topic Theme
    topic_theme = topic

    # 1. Author ID
    author_id = row['author_id']

    # 2. Time created
    try:
      created_at = dateutil.parser.parse(row['created_at'])
    except TypeError:
      created_at = row['created_at']

    # 3. Geolocation
    geo = row['geo']

    # 4. Tweet ID
    tweet_id = row['id']

    # 5. Language
    lang = row['lang']

    # 6. Tweet metrics
    # transform string to dictionary
    try:
      public_metrics = ast.literal_eval(row['public_metrics'])
      
      retweet_count = public_metrics['retweet_count']
      reply_count = public_metrics['reply_count']
      like_count = public_metrics['like_count']
      quote_count = public_metrics['quote_count']
    except ValueError:
      retweet_count = "nan"
      reply_count = "nan"
      like_count = "nan"
      quote_count = "nan"


    # 7. source
    source = row['source']

    # 8. Tweet text
    text = row['text']

    # 9. Retweeted
    try:
      referenced_tweets = ast.literal_eval(row['referenced_tweets'])
    except ValueError:
      retweeted = referenced_tweets

    retweeted = referenced_tweets[0]['type']
    
    # Assemble all data in a list
    res = [topic_theme, author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text, retweeted]
    
    if index == 0:
      # first time, so create dataframe from list
      df = pd.DataFrame([res], columns = ["topic_theme", "author_id", "created_at", "geo", "tweet_id", "lang", "like_count", "quote_count", "reply_count", "retweet_count", "source", "text", "retweeted"])
    else:
      df_length = len(df)
      if df_length % 1000 == 0:
        # log where we are, every 1000s entry
        print(df_length)
      df.loc[df_length] = res

1000
2000
3000
4000


In [ ]:
# check new preprocessed data frame
df

,topic_theme,author_id,created_at,geo,tweet_id,lang,like_count,quote_count,reply_count,retweet_count,source,text,retweeted
0,ukraine conflict,1179464402791219206,2022-01-02 19:03:19+00:00,NaN,1477717156959858693,en,0,0,1,0,Twitter for iPhone,deal that temporarily resolved the long-standi...,replied_to
1,ukraine conflict,1251224826984804352,2022-01-02 18:59:21+00:00,NaN,1477716157264121856,en,1,0,0,0,Twitter Web App,US pledges to help Ukraine shore up border wit...,replied_to
2,ukraine conflict,950864773087932416,2022-01-02 18:57:39+00:00,NaN,1477715730376400901,en,0,0,0,30,Twitter Web App,RT @EuromaidanPR: Russian hybrid forces launch...,retweeted
3,ukraine conflict,104700877,2022-01-02 18:53:40+00:00,NaN,1477714726511472641,en,0,0,0,0,Twitter for iPhone,Russia-Ukraine Conflict Lies in the Bones of a...,retweeted
4,ukraine conflict,1037420964320104450,2022-01-02 18:41:02+00:00,NaN,1477711547707404291,en,0,0,1,0,Twitter Web App,They're looking for his bones.\nThe Russia-Ukr...,retweeted
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4401,ukraine conflict,2787089534,2021-12-26 19:39:54+00:00,NaN,1475189647252807684,en,0,0,0,25,Twitter for Android,RT @PrplScrubsCHPN: @RonFilipkowski @Judson4Co...,retweeted
4402,ukraine conflict,3053193701,2021-12-26 19:38:24+00:00,NaN,1475189268989554690,en,0,0,0,28,Twitter for iPad,RT @SkyNews: Speaking at his annual news confe...,retweeted
4403,ukraine conflict,122756897,2021-12-26 19:35:44+00:00,NaN,1475188601403150336,en,0,0,1,0,Twitter for Android,@Morearighty It's easy. I'm a Russian immigran...,replied_to
4404,ukraine conflict,1400022519613214720,2021-12-26 19:34:58+00:00,NaN,1475188408427421701,en,0,0,0,19,Twitter for Android,RT @France24_en: @RobParsonsF24 The Wagner Gro...,retweeted


In [ ]:
# check length - should be the same as for d
len(df)

4406

In [ ]:
# should be true
len(df) == len(d)

True

In [ ]:
# create new file for preprocessed data - and ensure the name fits to a certain degree with the input file
text_file_name = "preprocessed_" + str(topic) + ".csv"
text_file_name

'preprocessed_ukraine conflict.csv'

In [ ]:
# send file to local storage where it can be further processed
df.to_csv(text_file_name)